In [25]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction import FeatureHasher

In [46]:
def get_features_titulo():
    return 20

def get_clusters_avisos():
    return 10
    
def get_avisos_detalle():
    #avisos1 = pd.read_csv('data/fiuba_6_avisos_detalle.csv')
    #avisos2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_6_avisos_detalle.csv')
    #avisos3 = pd.read_csv('data/FiubaHasta15Abril/fiuba_6_avisos_detalle.csv')
    #avisos4 = pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')        
    avisos_detalle = pd.read_csv('data/fiuba_6_avisos_detalle.csv')# pd.concat([avisos1,avisos2,avisos3,avisos4]).drop_duplicates(subset=['idaviso'], keep='last').reset_index(drop=True)
    columns_rename = {'idpostulante': 'id_postulante', 'idaviso': 'id_aviso'}
    avisos_detalle = avisos_detalle.rename(columns=columns_rename)
    to_nivel_laboral_nro = {'Senior / Semi-Senior' : 2, 'Junior':1, 'Otro':0,
       'Jefe / Supervisor / Responsable':3,
       'Gerencia / Alta Gerencia / Dirección':4}
    to_tipo_trabajo_nro={'Full-time':0, 'Part-time':1, 'Teletrabajo':2, 'Por Horas':3, 'Pasantia':4,
       'Temporario':5, 'Por Contrato':6, 'Fines de Semana':7, 'Primer empleo':8,
       'Voluntario':9}
    to_nombre_area_numero = pd.Series(avisos_detalle['nombre_area'].unique()).to_dict()
    to_nombre_area_numero  = {v: k for k, v in to_nombre_area_numero.items()}
    to_nombre_zona_numero = pd.Series(avisos_detalle['nombre_zona'].unique()).to_dict()
    to_nombre_zona_numero  = {v: k for k, v in to_nombre_zona_numero.items()}
    avisos_detalle['nivel_laboral_nro']= avisos_detalle['nivel_laboral'].map(to_nivel_laboral_nro)
    avisos_detalle['tipo_de_trabajo_nro'] = avisos_detalle['tipo_de_trabajo'].map(to_tipo_trabajo_nro)
    avisos_detalle['nombre_area_nro'] = avisos_detalle['nombre_area'].map(to_nombre_area_numero)
    avisos_detalle['zona'] = avisos_detalle['nombre_zona'].map(to_nombre_zona_numero)
    avisos_detalle['titulo_texto'] = avisos_detalle['titulo']
    h = FeatureHasher(n_features = get_features_titulo(), non_negative=True, input_type='string', dtype='int')
    x = h.transform(avisos_detalle['titulo'])
    avisos_detalle['titulo'] = list(x.toarray())

    titulos_como_lista = avisos_detalle.titulo.apply(pd.Series)
    avisos_detalle = pd.merge(avisos_detalle, titulos_como_lista, left_index = True, right_index = True)
    avisos_detalle = avisos_detalle.drop(['titulo'], axis=1)    
    
    
    return avisos_detalle

def x_avisos():
    return ['tipo_de_trabajo_nro', 'nivel_laboral_nro', 'nombre_area_nro', 'zona'] + list(range(0,get_features_titulo()))

def columnas_para_leer():
    return ['descripcion', 'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'titulo_texto','nombre_area','denominacion_empresa', 'cluster']

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

plt.figure(figsize=(12, 12))

n_samples = 1500
random_state = 170
X, y = make_blobs(n_samples=n_samples, random_state=random_state)

# Incorrect number of clusters
y_pred = KMeans(n_clusters=2, random_state=random_state).fit_predict(X)

In [3]:
X

array([[ -5.19811282e+00,   6.41869316e-01],
       [ -5.75229538e+00,   4.18627111e-01],
       [ -1.08448984e+01,  -7.55352273e+00],
       ..., 
       [  1.36105255e+00,  -9.07491863e-01],
       [ -3.54141108e-01,   7.12241630e-01],
       [  1.88577252e+00,   1.41185693e-03]])

In [47]:
avisos = get_avisos_detalle()
avisos.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\feature_extraction\hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


,id_aviso,idpais,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,...,10,11,12,13,14,15,16,17,18,19
0,8725750,1,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR,...,0,0,4,0,2,0,0,0,1,3
1,17903700,1,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste,...,1,0,0,0,0,0,1,0,0,1
2,1000150677,1,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL,...,1,0,2,0,0,0,0,0,0,2
3,1000610287,1,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A,...,0,0,7,0,2,0,0,0,1,0
4,1000872556,1,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,...,3,0,8,0,0,1,1,2,0,1


In [48]:
X = avisos[x_avisos()]

In [49]:
X.head()

,tipo_de_trabajo_nro,nivel_laboral_nro,nombre_area_nro,zona,0,1,2,3,4,5,...,10,11,12,13,14,15,16,17,18,19
0,0,2,0,0,0,3,6,0,0,0,...,0,0,4,0,2,0,0,0,1,3
1,0,2,1,0,0,0,0,1,1,2,...,1,0,0,0,0,0,1,0,0,1
2,0,2,2,1,0,2,0,3,0,0,...,1,0,2,0,0,0,0,0,0,2
3,0,2,2,0,0,9,9,0,0,0,...,0,0,7,0,2,0,0,0,1,0
4,0,2,3,0,0,4,1,5,1,4,...,3,0,8,0,0,1,1,2,0,1


In [50]:
y_pred = KMeans(n_clusters=get_clusters_avisos(), random_state=random_state).fit_predict(X)

In [51]:
avisos['cluster'] = y_pred

In [43]:
avisos.columns

Index([            'id_aviso',               'idpais',          'descripcion',
                'nombre_zona',               'ciudad',            'mapacalle',
            'tipo_de_trabajo',        'nivel_laboral',          'nombre_area',
       'denominacion_empresa',                 'zona',                      0,
                            1,                      2,                      3,
                            4,                      5,                      6,
                            7,                      8,                      9,
                           10,                     11,                     12,
                           13,                     14,                     15,
                           16,                     17,                     18,
                           19,              'cluster'],
      dtype='object')

In [54]:
avisos[avisos['cluster']==5][columnas_para_leer()]

,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,titulo_texto,nombre_area,denominacion_empresa,cluster
18,"<p>SC Johnson, empresa líder mundial en fabric...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Analista de Demand Planning,Abastecimiento,SC Johnson & Son de Argentina,5
19,<p>Centro Médico Accord se encuentra en la bús...,Gran Buenos Aires,Full-time,Junior,Administrativo de Recepción,Administración,Unión Personal,5
20,<p>Nos encontramos en la búsqueda de un Analis...,Gran Buenos Aires,Full-time,Junior,Analista Contable,Contabilidad,Motomel S.A.,5
21,<p>Importante correo privado ubicado en SAN MI...,Gran Buenos Aires,Full-time,Junior,Distribuidor domiciliario con moto (SAN MIGUEL),Distribución,Asoko Tempo SA,5
22,<p><strong>Express se encuentra en la búsqueda...,Gran Buenos Aires,Full-time,Otro,OPERADOR para Centro Operaciones de Red,Redes,TELEDIFUSORA SA,5
25,"<p style=""""><span style="""">Somos una empresa a...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,TECNICO SSr. -Soporte a usuarios nivel 2-,Soporte Técnico,Grupo Pujol Figueras,5
27,<p>Somos una empresa 100% argentina que se esp...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Jefe de Calidad con movilidad,Calidad,Food Service,5
29,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,SUPERVISOR COMODORO RIVADAVIA/BAHIA BLANCA,Oficios y Profesiones,Wurth Argentina S.A,5
34,<p>Importante laboratorio farmacéutico naciona...,Capital Federal,Full-time,Senior / Semi-Senior,Checker de procesos (laboratorio),Farmacéutica,GI GROUP Argentina,5
35,<p>Acciona IT se encuentra en la búsqueda de P...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,PM. Zona Norte / Pacheco,Liderazgo de Proyecto,ACCIONA IT,5
